<a href="https://colab.research.google.com/github/Tanzaniav0825/Algorithms-of-Data-Science/blob/session-5/Deliverable_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 🚀 One-cell clean deploy to your HF Space
!pip install -q fastapi uvicorn[standard] huggingface_hub pydantic httpx

from huggingface_hub import HfFolder, upload_folder
from getpass import getpass
import os, shutil, textwrap, time, requests

# 1) Auth
tok = HfFolder.get_token()
if not tok:
    tok = getpass("🔑 Enter your Hugging Face write token: ")
    HfFolder.save_token(tok)

SPACE_ID = "Tanzania0825/credibility-space"
BASE = "https://" + SPACE_ID.lower().replace("/", "-") + ".hf.space"

# 2) Build clean folder structure
base = "/content/credibility-chatbot"
if os.path.exists(base): shutil.rmtree(base)
os.makedirs(f"{base}/credscore/service", exist_ok=True)
os.makedirs(f"{base}/credscore/model", exist_ok=True)

# package __init__.py
for rel in ["__init__.py", "service/__init__.py", "model/__init__.py"]:
    open(f"{base}/credscore/{rel}", "w").close()

# service app (health + scoring)
open(f"{base}/credscore/service/app.py","w").write(textwrap.dedent("""
from fastapi import FastAPI
from pydantic import BaseModel
import time

app = FastAPI(title="Credibility Scoring Service", version="1.0.0")

class Citation(BaseModel):
    url: str
    title: str | None = None
    author: str | None = None
    published_at: str | None = None

class ScoreRequest(BaseModel):
    query: str
    answer: str
    citations: list[Citation] = []
    return_explanations: bool = True
    timeout_ms: int = 1800

@app.get("/health")
async def health():
    return {"ok": True}

@app.post("/v1/credibility/score")
async def score(req: ScoreRequest):
    t0 = time.perf_counter()
    prior = 0.6 if req.citations else 0.5
    content = 0.5 + min(0.4, 0.002 * len(req.answer))
    p = 0.5 * prior + 0.5 * content
    return {
        "score": p,
        "label": "high" if p > 0.8 else "medium" if p > 0.6 else "low",
        "confidence_interval": [max(0.0, p - 0.15), min(1.0, p + 0.15)],
        "explanations": None,
        "warnings": [],
        "latency_ms": int((time.perf_counter() - t0) * 1000),
    }
"""))

# Space entrypoint imports your app and adds /space_health
open(f"{base}/app.py","w").write(textwrap.dedent("""
from credscore.service.app import app

@app.get("/space_health")
async def space_health():
    return {"ok": True, "source": "space"}
"""))

# Minimal, stable deps
open(f"{base}/requirements.txt","w").write("fastapi==0.110.0\nuvicorn[standard]==0.27.1\npydantic>=2.7,<3.0\nhttpx>=0.27,<1.0\n")

# Dockerfile (shell-form CMD so port works)
open(f"{base}/Dockerfile","w").write(textwrap.dedent("""
FROM python:3.11-slim
ENV DEBIAN_FRONTEND=noninteractive PYTHONUNBUFFERED=1 PIP_NO_CACHE_DIR=1
WORKDIR /app
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt
COPY . .
ENV PYTHONPATH=/app
ENV PORT=7860
EXPOSE 7860
CMD python -m uvicorn app:app --host 0.0.0.0 --port ${PORT:-7860}
"""))

print("📦 Built clean project at:", base)

# 3) Upload to Space
upload_folder(repo_id=SPACE_ID, folder_path=base, path_in_repo=".", repo_type="space", token=tok)
print("📤 Uploaded to Space. Waiting ~45s for RUNNING...")

time.sleep(45)

# 4) Verify endpoints
def probe(path):
    try:
        r = requests.get(BASE + path, timeout=30)
        print(path, "->", r.status_code, r.headers.get("content-type",""))
        print((r.text[:200] if r.status_code != 200 else r.text)[:200])
    except Exception as e:
        print(path, "error:", e)

probe("/health")
probe("/openapi.json")

# 5) Quick scoring test
try:
    payload = {
      "query":"Is turmeric anti-inflammatory?",
      "answer":"Several studies suggest turmeric has anti-inflammatory effects.",
      "citations":[{"url":"https://www.nih.gov"}],
      "return_explanations": True,
      "timeout_ms": 1800
    }
    r = requests.post(BASE + "/v1/credibility/score", json=payload, timeout=45)
    print("/v1/credibility/score ->", r.status_code, r.headers.get("content-type",""))
    print(r.text[:500])
except Exception as e:
    print("score call error:", e)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.7/517.7 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.8/456.8 kB 30.5 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


🔑 Enter your Hugging Face write token: ··········
📦 Built clean project at: /content/credibility-chatbot
📤 Uploaded to Space. Waiting ~45s for RUNNING...
/health -> 200 application/json
{"ok":true}
/openapi.json -> 200 application/json
{"openapi":"3.1.0","info":{"title":"Credibility Scoring Service","version":"1.0.0"},"paths":{"/health":{"get":{"summary":"Health","operationId":"health_health_get","responses":{"200":{"description":"S
/v1/credibility/score -> 200 application/json
{"score":0.613,"label":"medium","confidence_interval":[0.46299999999999997,0.763],"explanations":null,"warnings":[],"latency_ms":0}
